In [323]:
import pandas as pd
from datetime import timedelta, datetime
import datetime as dt
import time
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Polyline, Popup, basemaps, AwesomeIcon
from IPython.display import display, clear_output


In [324]:
# Load GTFS data
data_path = r"C:\Users\TULPAR\JupyterLab Projects\GTFS Project\dataset"
stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")
shapes = pd.read_csv(f"{data_path}/shapes.txt")


In [325]:
# Function to convert the time of the data
def convert_time(time_str):
    try:
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        if hours >= 24:
            hours -= 24
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

# Apply the conversion to arrival_time and departure_time
stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])


In [326]:
# Simulation parameters
def get_time_input(prompt):
    while True:
        try:
            time_str = input(prompt)
            # Validate and convert input to time object
            time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
            return time_obj
        except ValueError:
            print("Invalid time format. Please enter time in HH:MM:SS format.")


start_time = get_time_input("Enter start time (HH:MM:SS): ")
end_time = get_time_input("Enter end time (HH:MM:SS): ")
current_time=start_time
print(f"Simulation will run from {start_time} to {end_time}.")
print(type(current_time))




Simulation will run from 10:00:02 to 10:02:00.
<class 'datetime.time'>


In [327]:
def find_service_id(calendar):
    today = datetime.today().weekday() + 1
    for index in calendar.index:
        value = calendar.iloc[index, today]
        if value == 1:
            servid = calendar.iloc[index, 0]
            return servid
    return None

# The service_id of the trip for today          
servid = find_service_id(calendar)
print(servid)

6


In [328]:
#Initialize the map centered on the city
center=(stops['stop_lat'].mean(),stops['stop_lon'].mean())
map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom_start=12)

In [329]:
# Add stop markers to the map initially
def add_stop_markers(stops, map):
    for stop in stops.itertuples():
        # Create a marker
        marker = Marker(location=[stop.stop_lat, stop.stop_lon], draggable=False)
        # Create a popup with the stop information
        popup_content = HTML()
        popup_content.value = f"Stop: {stop.stop_name}, Stop Number: {stop.stop_id}"
        popup = Popup(
            location=[stop.stop_lat, stop.stop_lon],
            child=popup_content,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )
        # Add the popup to the marker
        marker.popup = popup

        # Add the marker to the map
        map.add_layer(marker)
    return map
map=add_stop_markers(stops,map)

In [330]:
def create_lines(shapes, map):
    shape_locations = []
    for shape in shapes.itertuples():
        shape_locations.append([shape.shape_pt_lat, shape.shape_pt_lon])
    
    if len(shape_locations) > 1:
        polyline = Polyline(
            locations=shape_locations,
            color="red",
            fill=False,
            weight=2,
            opacity=1,
            dash_array='5 ,5'
        )
        map.add_layer(polyline)
    
    return map
map=create_lines(shapes,map)


In [336]:
def initialize_buses(trips,stop_times,stops,routes):
    # Dictionary to store bus markers
    merged_df = pd.merge(trips, stop_times, on='trip_id', how='outer')
    merged_df2=pd.merge(merged_df,stops, on='stop_id', how='outer')
    final_merge=pd.merge(merged_df2,routes, on='route_id', how='outer')

    # Initialize buses
    buses = [{
        'service_id': trip.service_id,
        'trip_id': trip.trip_id,
        'route_id': trip.route_id,
        'current_stop_sequence': trip.stop_sequence,
        'stop_time': trip.arrival_time,
        'direction_id': trip.direction_id,
        'current_stop_id': trip.stop_id,
        'current_stop_name': trip.stop_name,
        'stop_lat': trip.stop_lat,
        'stop_lon': trip.stop_lon,
        'route_short_name': trip.route_short_name
        
    } for trip in final_merge.itertuples()]
    return buses

buses=initialize_buses(trips,stop_times,stops,routes)


In [335]:
def filter_buses(buses,servid,current_time):
    filtered_buses = []
    #desired_time = datetime.strptime('10:00:00', '%H:%M:%S').time() curre
    for bus in buses:
        bus_service_id = bus['service_id']
        bus_stop_time = datetime.strptime(bus['stop_time'], '%H:%M:%S').time()
        
        if bus_service_id == servid and bus_stop_time == current_time:
            filtered_buses.append(bus)
            
    return filtered_buses

filtered_buses=filter_buses(buses,servid,current_time)
print(filtered_buses)



[]


In [333]:
def create_bus_markers(bus):
    icon=AwesomeIcon(name='bus', marker_color='red', icon_color='white')
    
    # Create a marker 
    marker = Marker(icon=icon, location=[bus['stop_lat'], bus['stop_lon']], draggable=False)

    # Create a popup with the stop information
    popup_content = HTML()
    popup_content.value = f"Route Name: {bus['route_short_name']},Trip ID: {bus['trip_id']},Stop Name: {bus['current_stop_name']}, Stop Number: {bus['current_stop_id']},Stop Time: {bus['stop_time']}"
    popup = Popup(
        location=[bus['stop_lat'],bus['stop_lon']],
        child=popup_content,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

    # Add the popup to the marker
    marker.popup = popup
    return marker


    


In [334]:

def update_buses(buses, servid, start_time, end_time, map):
    current_time = start_time
    existing_markers = {}
    previous_markers={}


    while current_time <= end_time:
        print(f"Current time: {current_time}")
        # Filter buses and get new marker locations
        filtered_buses = filter_buses(buses, servid, current_time)
        new_markers = {(bus['stop_lat'], bus['stop_lon']): bus for bus in filtered_buses}

         # Remove markers that are no longer valid
        for marker_key in list(existing_markers.keys()):
            if marker_key not in new_markers:
                map.remove_layer(existing_markers[marker_key])
                del existing_markers[marker_key]

        # Add new markers or update existing ones
        for marker_key, bus in new_markers.items():
            if marker_key not in existing_markers:
                # Add new marker
                marker = create_bus_markers(bus)
                map.add_layer(marker)
                existing_markers[marker_key] = marker
            else:
                # Update existing marker if location has changed
                old_marker = existing_markers[marker_key]
                new_location = [bus['stop_lat'], bus['stop_lon']]
                if old_marker.location != new_location:
                    map.remove_layer(old_marker)
                    new_marker = create_bus_markers(bus)
                    map.add_layer(new_marker)
                    existing_markers[marker_key] = new_marker

        # Increment time by 1 second
        current_time = (datetime.combine(datetime.today(), current_time) + timedelta(seconds=1)).time()
        # Sleep for 1 second to simulate time passing
        time.sleep(1)

newbus=update_buses(buses,servid, start_time, end_time, map)
print(newbus)

Current time: 10:00:02
Current time: 10:00:03
Current time: 10:00:04
Current time: 10:00:05
Current time: 10:00:06
Current time: 10:00:07
Current time: 10:00:08


KeyboardInterrupt: 

In [ ]:
'''# Simulation loop
while current_time <= end_time:
    filtered_buses = filter_buses(buses)  # Ensure filtered_buses is updated
    updated_buses = update_buses(filtered_buses, stop_times, current_time)
    
    # Clear previous bus markers and add updated ones
    map.layers = [layer for layer in map.layers if not isinstance(layer, Marker)]  # Remove old markers
    create_bus_markers(updated_buses, map)
    
    clear_output(wait=True)
    display(map)  # Use display(map) to show updated map
    
    # Update the current time
    current_time = (datetime.combine(datetime.today(), current_time) + time_step).time()
    print(f"Current time: {current_time.strftime('%H:%M:%S')}")
    
    # Optional: sleep for visual update
    # time.sleep(1)
'''